# Chess Move Prediction Model
This notebook demonstrates how to preprocess a chess dataset and train a neural network to predict the next move given the current board state.

In [ ]:
import pandas as pd
import numpy as np
import chess
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = '/content/games.csv'  # Upload the dataset to Google Colab
chess_data = pd.read_csv(file_path)

# Function to encode chess board state into a 64-element vector
piece_to_int = {
    None: 0,
    chess.PAWN: 1,
    chess.KNIGHT: 2,
    chess.BISHOP: 3,
    chess.ROOK: 4,
    chess.QUEEN: 5,
    chess.KING: 6,
}

def encode_board(fen):
    board = chess.Board(fen)
    encoded_board = np.zeros(64, dtype=int)
    for i in range(64):
        piece = board.piece_at(i)
        if piece:
            value = piece_to_int[piece.piece_type]
            if piece.color == chess.WHITE:
                encoded_board[i] = value
            else:
                encoded_board[i] = -value  # Negative for black pieces
    return encoded_board

# Preprocess the `moves` column to generate board positions and moves
def preprocess_moves(moves_column):
    positions = []
    moves = []

    for moves_sequence in moves_column:
        board = chess.Board()
        move_sequence = moves_sequence.split()

        for i in range(len(move_sequence) - 1):  # Use all moves except the last
            positions.append(encode_board(board.fen()))
            moves.append(move_sequence[i])
            board.push_uci(move_sequence[i])

    return np.array(positions), np.array(moves)

# Using a subset of data for processing
sample_moves = chess_data['moves'][:1000]  # Adjust sample size as needed
positions, moves = preprocess_moves(sample_moves)

# Encode moves into numerical format
label_encoder = LabelEncoder()
encoded_moves = label_encoder.fit_transform(moves)
move_classes = len(label_encoder.classes_)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(positions, encoded_moves, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(64,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(move_classes, activation='softmax')
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")

# Save the model
model.save('chess_move_prediction_model.h5')

# Predict a move for a given board state
def predict_next_move(fen):
    encoded_board = encode_board(fen).reshape(1, -1)
    predictions = model.predict(encoded_board)
    predicted_move_index = np.argmax(predictions)
    return label_encoder.inverse_transform([predicted_move_index])[0]

# Example usage
test_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
predicted_move = predict_next_move(test_fen)
print(f"Predicted move: {predicted_move}")
